# Callbacks

In [ ]:
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("xray_model.h5", save_best_only=True)

early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    patience=10, restore_best_weights=True
)


# Weight imbalance

In [ ]:
initial_bias = np.log([COUNT_PNEUMONIA / COUNT_NORMAL])
print("Initial bias: {:.5f}".format(initial_bias[0]))

TRAIN_IMG_COUNT = COUNT_NORMAL + COUNT_PNEUMONIA
weight_for_0 = (1 / COUNT_NORMAL) * (TRAIN_IMG_COUNT) / 2.0
weight_for_1 = (1 / COUNT_PNEUMONIA) * (TRAIN_IMG_COUNT) / 2.0

class_weight = {0: weight_for_0, 1: weight_for_1}

print("Weight for class 0: {:.2f}".format(weight_for_0))
print("Weight for class 1: {:.2f}".format(weight_for_1))

# CNNs

In [3]:
import tensorflow as tf
print(f"TensorFlow version: {tf.__version__}")
#import keras (high level API) wiht tensorflow as backend
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, AveragePooling1D
from tensorflow.keras.callbacks import ModelCheckpoint

print(x_train.shape)
print(x_test.shape)

def build_cnn_model(input_shape):
    model = Sequential()
    
    # 2 Convolution layer with Max pooling
    model.add(Conv1D(filters=8, kernel_size=51, activation = 'relu', padding = 'same', input_shape = input_shape))
    model.add(AveragePooling1D(pool_size=10))
    model.add(Conv1D(filters=2, kernel_size=9, activation = 'relu', padding = 'same', kernel_initializer = "he_normal"))
    model.add(AveragePooling1D())  
    model.add(Flatten())
    
    # 3 Full connected layer
    model.add(Dense(16, activation = 'relu', kernel_initializer = "he_normal"))
    #model.add(Dense(32, activation = 'relu', kernel_initializer = "he_normal"))
    #model.add(Dense(54, activation = 'relu', kernel_initializer = "he_normal"))
    model.add(Dense(2, activation = 'softmax')) # 2 classes 
    
    # summarize the model
    print(model.summary())
    
    # Compile
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

def compile_and_fit_model(cnn_model, x_train, y_train, x_test, y_test):
    #history = cnn_model.fit(x_train,y_train, epochs=100, batch_size=64, verbose=2)
    #_, accuracy = cnn_model.evaluate(x_test, y_test, batch_size=64, verbose=2)
    history = cnn_model.fit(x_train, y_train, validation_data=(x_test,y_test), epochs=250, batch_size=128, verbose=0)
    return cnn_model, history

input_shape = (x_train.shape[1],1)
cnn_model = build_cnn_model(input_shape)

# train cnn model
x_train_reshaped = np.array(x_train).reshape((x_train.shape[0],x_train.shape[1],1))
x_test_reshaped = np.array(x_test).reshape((x_test.shape[0],x_test.shape[1],1))
trained_cnn_model, cnn_history = compile_and_fit_model(cnn_model, x_train_reshaped, y_train, x_test_reshaped, y_test)


plt.figure(figsize=(18,8))
plt.subplot(1,2,1)
plt.plot(cnn_history.history['loss'],label='Loss')
plt.plot(cnn_history.history['val_loss'],label='Validation Loss')
plt.legend(loc='best')
plt.subplot(1,2,2)
plt.plot(cnn_history.history['accuracy'],label='Accuracy')
plt.plot(cnn_history.history['val_accuracy'],label='Validation Accuracy')
plt.legend(loc='best')
plt.show()

TensorFlow version: 2.1.0


NameError: name 'x_train' is not defined

In [1]:
def evaluate_model_1D_CNN(trainX, trainy, testX, testy):
    epochs = 25 
    batch_size = 32
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features))) 
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100,activation='relu'))
    model.add(Dense(n_outputs,activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # Fit
    history = model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=2)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=2)
    return history, accuracy

In [2]:
def evaluate_model_multihead_CNN(trainX, trainy, testX, testy):
    epochs = 10
    batch_size = 32
    n_timesteps, n_features, n_outputs = x_train_raw.shape[1], x_train_raw.shape[2], y_train.shape[1]
    
    # head 1
    inputs1 = Input(shape=(n_timesteps, n_features))
    conv1 = Conv1D(filters=64, kernel=3, activation='relu')(inputs1)
    drop1 = Dropout(0.5)(conv1)
    pool1 = MaxPooling1D(pool_size=2)(drop1)
    flat1 = Flatten()(pool1)
    
    # head 2
    inputs2 = Input(shape=(n_timesteps, n_features))
    conv2 = Conv1D(filters=64, kernel=3, activation='relu')(inputs2)
    drop2 = Dropout(0.5)(conv2)
    pool2 = MaxPooling1D(pool_size=2)(drop2)
    flat2 = Flatten()(pool2)
    
    # head 3
    inputs3 = Input(shape=(n_timesteps, n_features))
    conv3 = Conv1D(filters=64, kernel=3, activation='relu')(inputs3)
    drop3 = Dropout(0.5)(conv3)
    pool3 = MaxPooling1D(pool_size=2)(drop3)
    flat3 = Flatten()(pool3)
    
    # merge
    merged = concatenate([flat1,flat2,flat3])
    
    # interpretation
    dense = Dense(100, activaition='relu')(merged)
    outputs = Dense(n_outputs, activation='softmax')(dense)
    model = Model(inputs=[inputs1,inputs2,inputs3], outputs=outputs)
    
    # compile 
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit
    history = model.fit([trainX,trainX,trainX], trainy, epochs=epochs, batch_size=batch_size, verbose=2)
    # evaluate model
    _, accuracy = model.evaluate([testX,testX,testX], testy, batch_size=batch_size, verbose=2)
    plot_model(model, to_file='multihead_CNN_example.png')
    return history, accuracy

# LSTMs

In [4]:
# Vanilla LSTM
def evaluate_model_LSTM(x_train_raw, y_train, x_test_raw, y_test):
    n_timesteps, n_features, n_outputs = x_train_raw.shape[1], x_train_raw.shape[2], y_train.shape[1]
    epochs = 25
    batch_size = 64
    # Define LSTM model
    model = Sequential()
    model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation = 'softmax')) # Output
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # Fit
    history = model.fit(x_train_raw,y_train, epochs=epochs, batch_size=batch_size, verbose=2)
    # Evaluation
    results, accuracy = model.evaluate(x_test_raw, y_test, batch_size=batch_size, verbose=2)
    return history, accuracy

'''
The CNN LSTM involves CNN layers to extracte the features on input data
combined with LSTM to support sequence prediction.

The CNN model will read subsequences of the main sequence in as blocks,
extract features from each block, then allow the LSTM to interpret the
features extracted from each block.

One approach to implementing this model is to split each window of time steps
into smaller subsequences for the CNN to process. (here 4x32=128)

It is common to use two consecutive CNN layers followed by dropout and
a maxpooling layer, which is the simple structure used here. Indeed, Conv layers
summarize the presence of features in an input image. The problem is that those
are sensitive to the location of the features in the input. One approach to 
address this sensitivity is to down sample the feature maps. This has the 
effect of making the resulting down sampled feature maps more robust to changes 
in the position of the feature in the image, referred to by the technical phrase
"local translation invariance". Pooling layers provide an approach to down
sampling feature maps by summarizing the presence of features in patches
of the feature map. Two common pooling methods are average pooling and max pooling
to summarize the average presence of a feature and the most actived presence
of a feature respectively.

Kernel_size = 3 (means you lose two datapoints on the next layer)
filters = 64 (means the batch size actually)
pool_size = 2 (means the size of feature map reduced by a factor of 2, almost always)


In [5]:
def evaluate_model_CNN_LSTM(x_train_raw, y_train, x_test_raw, y_test):
    _, n_features, n_outputs = x_train_raw.shape[1], x_train_raw.shape[2], y_train.shape[1]
    epochs = 50
    batch_size = 64
    # Reshape in subsequences for the CNN
    n_subs, n_lengths = 4, 32
    x_train_raw = x_train_raw.reshape(x_train_raw.shape[0], n_subs, n_lengths, n_features)
    x_test_raw = x_test_raw.reshape(x_test_raw.shape[0], n_subs, n_lengths, n_features)
    # Define CNN LSTM model
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=batch_size, kernel_size=3, activation='relu'), input_shape=(None,n_lengths,n_features)))
    model.add(TimeDistributed(Conv1D(filters=batch_size, kernel_size=3, activation='relu')))
    model.add(TimeDistributed(Dropout(0.5)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(100))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # Fit
    history = model.fit(x_train_raw, y_train, epochs=epochs, batch_size=batch_size, verbose = 2)
    # Evaluate model
    _, accuracy = model.evaluate(x_test_raw,y_test, batch_size=batch_size, verbose = 2)
    return history, accuracy

'''
A further extension of the CNN LSTM idea is to perform the convolutions of the 
CNN (e.g. how the CNN reads the input sequence data) as part of the LSTM.

Unlike an LSTM that reads the data in directly in order to calculate internal 
state and state transitions, and unlike the CNN LSTM that is interpreting the 
output from CNN models, the ConvLSTM is using convolutions directly as part of 
reading input into the LSTM units themselves.

For this chosen framing of the problem, the input for the ConvLSTM2D would therefore be:
reshape into subsequences (samples, time steps, rows, cols, channels)

Samples: n, for the number of windows in the dataset.
Time: 4, for the four subsequences that we split a window of 128 time steps into.
Rows: 1, for the one-dimensional shape of each subsequence.
Columns: 32, for the 32 time steps in an input subsequence.
Channels: 9, for the nine input variables.

'''

In [6]:
def evaluate_model_ConvLSTM(x_train_raw, y_train, x_test_raw, y_test):
    _, n_features, n_outputs = x_train_raw.shape[1], x_train_raw.shape[2], y_train.shape[1]
    epochs = 50
    batch_size = 64
    # Reshaping of the ConvLSTM
    n_steps, n_length = 4, 32
    x_train_raw = x_train_raw.reshape((x_train_raw.shape[0], n_steps, 1, n_length, n_features))
    x_test_raw = x_test_raw.reshape((x_test_raw.shape[0], n_steps, 1, n_length, n_features))
	# define model
    model = Sequential()
    model.add(ConvLSTM2D(filters=64, kernel_size=(1,3), activation='relu', input_shape=(n_steps, 1, n_length, n_features)))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	# fit network
    history = model.fit(x_train_raw, y_train, epochs=epochs, batch_size=batch_size, verbose=2)
	# evaluate model
    _, accuracy = model.evaluate(x_test_raw, y_test, batch_size=batch_size, verbose=2)
    return history, accuracy


In [7]:
# fit an LSTM network to training data
def fit_lstm(train, test, raw, scaler, batch_size, nb_epoch, neurons):
	X, y = train[:, 0:-1], train[:, -1]
	X = X.reshape(X.shape[0], 1, X.shape[1])
	# prepare model
	model = Sequential()
	model.add(LSTM(neurons, batch_input_shape=(batch_size, X.shape[1], X.shape[2]), stateful=True))
	model.add(Dense(1))
	model.compile(loss='mean_squared_error', optimizer='adam')
	# fit model
	train_rmse, test_rmse = list(), list()
	for i in range(nb_epoch):
		model.fit(X, y, epochs=1, batch_size=batch_size, verbose=0, shuffle=False)
		model.reset_states()
		# evaluate model on train data
		raw_train = raw[-(len(train)+len(test)+1):-len(test)]
		train_rmse.append(evaluate(model, raw_train, train, scaler, 0, batch_size))
		model.reset_states()
		# evaluate model on test data
		raw_test = raw[-(len(test)+1):]
		test_rmse.append(evaluate(model, raw_test, test, scaler, 0, batch_size))
		model.reset_states()
	history = DataFrame()
	history['train'], history['test'] = train_rmse, test_rmse
	return history

In [9]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

def conv_block(filters, inputs):
    x = layers.SeparableConv2D(filters, 3, activation="relu", padding="same")(inputs)
    x = layers.SeparableConv2D(filters, 3, activation="relu", padding="same")(x)
    x = layers.BatchNormalization()(x)
    outputs = layers.MaxPool2D()(x)

    return outputs

def dense_block(units, dropout_rate, inputs):
    x = layers.Dense(units, activation="relu")(inputs)
    x = layers.BatchNormalization()(x)
    outputs = layers.Dropout(dropout_rate)(x)

    return outputs

def build_model():
    inputs = keras.Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))
    x = preprocessing.Rescaling(1.0 / 255)(inputs)
    x = layers.Conv2D(16, 3, activation="relu", padding="same")(x)
    x = layers.Conv2D(16, 3, activation="relu", padding="same")(x)
    x = layers.MaxPool2D()(x)

    x = conv_block(32, x)
    x = conv_block(64, x)

    x = conv_block(128, x)
    x = layers.Dropout(0.2)(x)

    x = conv_block(256, x)
    x = layers.Dropout(0.2)(x)

    x = layers.Flatten()(x)
    x = dense_block(512, 0.7, x)
    x = dense_block(128, 0.5, x)
    x = dense_block(64, 0.3, x)

    outputs = layers.Dense(1, activation="sigmoid")(x)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

# Autoencoders

In [8]:
model = keras.Sequential(
    [
        layers.Input(shape=(x_train.shape[1], x_train.shape[2])),
        layers.Conv1D(
            filters=32, kernel_size=7, padding="same", strides=2, activation="relu"
        ),
        layers.Dropout(rate=0.2),
        layers.Conv1D(
            filters=16, kernel_size=7, padding="same", strides=2, activation="relu"
        ),
        layers.Conv1DTranspose(
            filters=16, kernel_size=7, padding="same", strides=2, activation="relu"
        ),
        layers.Dropout(rate=0.2),
        layers.Conv1DTranspose(
            filters=32, kernel_size=7, padding="same", strides=2, activation="relu"
        ),
        layers.Conv1DTranspose(filters=1, kernel_size=7, padding="same"),
    ]
)
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss="mse")
model.summary()

NameError: name 'layers' is not defined